<a href="https://colab.research.google.com/github/chiyanglin-AStar/2025_physics_note/blob/main/02_Pycuda_ex0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is reference from [Linking Python to CUDA with PyCUDA: A Beginner’s Guide](https://medium.com/@mahmoudalyosify/linking-python-to-cuda-with-pycuda-a-beginners-guide-d128da0ed460)

In [1]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.4/92.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.9 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660545 sha256=8641d8758cb68f4e649ebaa5c3a22df46104b2a5188f1605edc2b68ad8a4e4df
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [2]:
import pycuda.autoinit
import pycuda.driver as cuda

In [3]:
cuda.init()

In [4]:
device = cuda.Device(0)
context = device.make_context()

In [5]:
import numpy as np
from pycuda import driver, compiler, gpuarray

# Initialize PyCUDA
driver.init()

# Create a CUDA context
device = driver.Device(0)
context = device.make_context()

# Define the CUDA kernel
kernel_code = """
__global__ void add_arrays(float *a, float *b, float *c) {
    int i = threadIdx.x;
    c[i] = a[i] + b[i];
}
"""

# Compile the CUDA kernel
module = compiler.SourceModule(kernel_code)

# Allocate memory on the GPU
a_gpu = gpuarray.to_gpu(np.random.randn(100).astype(np.float32))
b_gpu = gpuarray.to_gpu(np.random.randn(100).astype(np.float32))
c_gpu = gpuarray.empty_like(a_gpu)

# Launch the CUDA kernel
add_arrays = module.get_function("add_arrays")
add_arrays(a_gpu, b_gpu, c_gpu, block=(100,1,1))

# Copy the result back to the CPU
c_cpu = c_gpu.get()

# Clean up
context.pop()